YAHOO News is a news aggregation website thats collects news from various sources

This notebook implements scraping Yahoo news using beautiful soup and newspaper module
Newspaper is a Python module used for extracting and parsing newspaper articles.

**References**:
* https://www.geeksforgeeks.org/newspaper-article-scraping-curation-python/
* https://github.com/israel-dryer/Yahoo-News-Scraper/blob/master/yahoo-news-scraper.ipynb

In [ ]:
pip install newspaper3k

     |████████████████████████████████| 215kB 4.4MB/s 
     |████████████████████████████████| 7.4MB 6.9MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398406 sha256=1511724f2f0e2b29d2727cf306513162c64bf96864ceef83de1a9eca6887c8ae
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13538 sha256=247e0f5884ccbb432d960c0ddcc4b55b8a2d11be531acad7be3c051d70255618
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=cb800fd87003cf2acaf69c276472f9e30081cee79cbe1bdb8c1fdd9446a7d6f2
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c75

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests
import sys
from newspaper import Article

headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline = card.find('h4', 's-title').text
    source = card.find("span", 's-source').text
    posted = card.find('span', 's-time').text.replace('·', '').strip()
    #description = card.find('p', 's-desc').text.strip()
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    clean_link = re.search(pattern, unquoted_link).group(1)
    #Creating an instance of the article
    parsed_article = Article(clean_link, language="en")
    #To download the article
    try:
      parsed_article.download()
      #To parse the article
      parsed_article.parse()
      #To perform natural language processing ie..nlp
      parsed_article.nlp()
    except:
      e = sys.exc_info()[0]
    article = (headline, source, posted, clean_link, parsed_article.summary)
    return article

def get_the_news(search):
    """Run the main program"""
    template = 'https://news.search.yahoo.com/search?p={}'
    url = template.format(search)
    articles = []
    links = set()
    
    for i in range(200):
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'NewsArticle')
        
        # extract articles from page
        for card in cards:
            article = get_article(card)
            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)        
                
        try:
            url = soup.find('a', 'next').get('href')
            sleep(1)
        except AttributeError:
            continue
            
    # save article data
    with open('results.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Headline', 'Source', 'Posted', 'Link', 'Summary'])
        writer.writerows(articles)
        
    return articles

In [ ]:
# run the main program
articles_final = []
news_list = ["corona","politics","stocks"]
for news in news_list:
  articles_final = get_the_news(news)

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1218: UnknownTim